In [ ]:
import pandas as pd
from pathlib import Path
import re
import shutil

In [ ]:
df = pd.read_csv('../data/Raw/Colones12.csv', encoding='latin1')
tarjetas = pd.read_json('tarjetas.json', orient='index')
df

In [ ]:
metadata = df.iloc[0:1, 0:4].drop(columns=['Número de Clientes'])

metadata = metadata.rename(columns=lambda x: x.strip())

metadata["Producto"] = metadata["Producto"].astype(str).str.strip()

mapeo = tarjetas.set_index('Cuenta IBAN Asociada CRC')['Nombre'].to_dict()

metadata['Producto'] = metadata['Producto'].map(mapeo)

transacciones = df.iloc[2:, 0:7].rename(columns=df.iloc[2]).iloc[2:].reset_index(drop=True)

transacciones = transacciones.iloc[:transacciones[transacciones['Fecha de Transacción'] == 'Resumen de Estado Bancario'].index[0]]

fecha = pd.to_datetime(transacciones["Fecha de Transacción"].iloc[0], dayfirst=True)

metadata['Fecha'] = fecha

metadata

In [ ]:
transacciones

In [ ]:
df = pd.read_csv('../data/OldRaw/Tarjeta_credito_cashback/Tarjeta_credito_cashback_02_2025_old.csv', encoding='latin1')
df

In [ ]:
header = {
    'Pro000000000000duct': 'Producto',
    ' Name': 'Nombre',
    ' Date': 'Fecha',
    ' Minimum payment/due date': 'Pago mínimo/fecha de vencimiento',
    ' Minimum payment/ Local Amount': 'Pago mínimo/Monto local',
    ' Minimum Payment / Dollars Amount': 'Pago mínimo/Monto en USD',
    ' Cash payment/Due date': 'Pago en efectivo/Fecha de vencimiento',
    ' Cash payment / Local amount': 'Pago en efectivo/Monto local',
    ' Cash payment / Dollar amount': 'Pago en efectivo/Monto en USD',
}

metadata = df.iloc[0:1, 0:9].rename(columns=header)

metadata['Saldo anterior'] = float(df.iloc[2:3, 2:3].values[0][0])

metadata['Producto'] = metadata['Producto'].astype(str)
metadata['first4'] = metadata['Producto'].str.split('-').str[0]
metadata['last4'] = metadata['Producto'].str.split('-').str[-1]
tarjetas['Numero de tarjeta'] = tarjetas['Numero de tarjeta'].astype(str)
tarjetas['first4'] = tarjetas['Numero de tarjeta'].str[:4]
tarjetas['last4'] = tarjetas['Numero de tarjeta'].str[-4:]

df_merged = metadata.merge(
    tarjetas[['first4', 'last4', 'Nombre']],
    on=['first4', 'last4'],
    how='left',
    suffixes=('', '_tarjeta')
)

df_merged['Producto'] = df_merged['Nombre_tarjeta']

metadata = df_merged.drop(columns=['Nombre_tarjeta', 'first4', 'last4'])

header = {
    'Pro000000000000duct': 'Fecha de Transacción',
    ' Name': 'Descripción',
    ' Date': 'Monton en CRC',
    ' Minimum payment/due date': 'Monto en USD',
}
transacciones = df.iloc[4:, 0:4].rename(columns=header).reset_index(drop=True)

transacciones['Fecha de Transacción'] = pd.to_datetime(transacciones['Fecha de Transacción'], 
                                            errors='coerce', 
                                            dayfirst=True)

transacciones = transacciones.dropna(subset=['Fecha de Transacción'])

#pos = transacciones[transacciones['Fecha de Transacción'].isnull()].index[0]
#transacciones = transacciones.iloc[:pos]

transacciones

In [ ]:
metadata

In [ ]:
transacciones

In [ ]:
def cleanup_debit(df, tarjetas):
    metadata = df.iloc[0:1, 0:4].drop(columns=['Número de Clientes'])
    
    metadata = metadata.rename(columns=lambda x: x.strip())
    metadata["Producto"] = metadata["Producto"].astype(str).str.strip()
    mapeo = tarjetas.set_index('Cuenta IBAN Asociada CRC')['Nombre'].to_dict()
    metadata['Producto'] = metadata['Producto'].map(mapeo)

    transacciones = df.iloc[2:, 0:7].rename(columns=df.iloc[2]).iloc[2:].reset_index(drop=True)

    transacciones = transacciones.iloc[:transacciones[transacciones['Fecha de Transacción'] == 'Resumen de Estado Bancario'].index[0]]
    
    fecha = pd.to_datetime(transacciones["Fecha de Transacción"].iloc[0], dayfirst=True)

    metadata['Fecha'] = fecha
    
    return metadata, transacciones, pd.to_datetime(metadata["Fecha"].iloc[0], dayfirst=True)

def cleanup_credit(df, tarjetas):
    header = {
    'Pro000000000000duct': 'Producto',
    ' Name': 'Nombre',
    ' Date': 'Fecha',
    ' Minimum payment/due date': 'Pago mínimo/fecha de vencimiento',
    ' Minimum payment/ Local Amount': 'Pago mínimo/Monto local',
    ' Minimum Payment / Dollars Amount': 'Pago mínimo/Monto en USD',
    ' Cash payment/Due date': 'Pago en efectivo/Fecha de vencimiento',
    ' Cash payment / Local amount': 'Pago en efectivo/Monto local',
    ' Cash payment / Dollar amount ': 'Pago en efectivo/Monto en USD',
}

    metadata = df.iloc[0:1, 0:9].rename(columns=header)

    metadata['Saldo anterior'] = float(df.iloc[2:3, 2:3].values[0][0])
    
    metadata['Producto'] = metadata['Producto'].astype(str)
    metadata['first4'] = metadata['Producto'].str.split('-').str[0]
    metadata['last4'] = metadata['Producto'].str.split('-').str[-1]
    tarjetas['Numero de tarjeta'] = tarjetas['Numero de tarjeta'].astype(str)
    tarjetas['first4'] = tarjetas['Numero de tarjeta'].str[:4]
    tarjetas['last4'] = tarjetas['Numero de tarjeta'].str[-4:]
    
    df_merged = metadata.merge(
        tarjetas[['first4', 'last4', 'Nombre']],
        on=['first4', 'last4'],
        how='left',
        suffixes=('', '_tarjeta')
    )
    
    df_merged['Producto'] = df_merged['Nombre_tarjeta']
    
    metadata = df_merged.drop(columns=['Nombre_tarjeta', 'first4', 'last4'])

    header = {
        'Pro000000000000duct': 'Fecha de Transacción',
        ' Name': 'Descripción',
        ' Date': 'Monton en CRC',
        ' Minimum payment/due date': 'Monto en USD',
    }
    transacciones = df.iloc[4:, 0:4].rename(columns=header).reset_index(drop=True)
    
    transacciones['Fecha de Transacción'] = pd.to_datetime(transacciones['Fecha de Transacción'], 
                                            errors='coerce', 
                                            dayfirst=True)
    
    transacciones = transacciones.dropna(subset=['Fecha de Transacción'])
    
    return metadata, transacciones, pd.to_datetime(metadata["Fecha"].iloc[0], dayfirst=True)

def cleanup_data(df, tarjetas):
    if 'Pro000000000000duct' in df.columns:
        return cleanup_credit(df, tarjetas)
    else:
        return cleanup_debit(df, tarjetas)

In [ ]:
base_path = Path('../data')
raw_path = base_path / 'Raw'
cleaned_path = base_path / 'cleanedData'
tarjetas = pd.read_json('tarjetas.json', orient='index')

cleaned_path.mkdir(exist_ok=True)

for file in raw_path.rglob('*'):
    if file.is_file():
        relative = file.relative_to(raw_path)
        full_path = raw_path / relative

        df = pd.read_csv(str(full_path), encoding='latin1')
        metadata, transacciones, fecha = cleanup_data(df, tarjetas)
        
        metadata.columns = metadata.columns.str.strip()

        card_id = str(metadata["Producto"].iloc[0])
        card_id_sanitized = re.sub(r'[\\/*?:"<>|]', "", card_id).replace(" ", "_")
        
        mes = fecha.strftime("%m")
        año = fecha.strftime("%Y")
        
        card_folder = cleaned_path / card_id_sanitized
        card_folder.mkdir(exist_ok=True)

        new_filename = f"{card_id_sanitized}_{mes}_{año}.csv"
        output_file = card_folder / new_filename
        transacciones.to_csv(output_file, index=False, encoding='utf-8')
        print("--------------------")
        print(f"Procesando archivo: {full_path}")
        print(f"Guardado transacciones: {output_file}")

        metadata_filename = f"metadata_{card_id_sanitized}_{mes}_{año}.csv"
        metadata_output_file = card_folder / metadata_filename
        metadata.to_csv(metadata_output_file, index=False, encoding='utf-8')
        print(f"Guardado metadata: {metadata_output_file}")
        
        old_raw_folder = base_path / 'OldRaw'
        old_raw_folder.mkdir(exist_ok=True)
        card_old_folder = old_raw_folder / card_id_sanitized
        card_old_folder.mkdir(exist_ok=True)
        
        original_filename = f"{card_id_sanitized}_{mes}_{año}_old{file.suffix}"
        destination = card_old_folder / original_filename
        shutil.move(str(full_path), str(destination))
        print(f"Archivo original movido a: {destination}")